<a href="https://colab.research.google.com/github/KamilShape/emotion_recognition/blob/main/playwright_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Photos download**

In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
!pip install playwright
!playwright install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 41.3 MB/s eta 0:00:00
146.4 Mb [] 0% 0.0s146.4 Mb [] 0% 28.2s146.4 Mb [] 0% 22.8s146.4 Mb [] 0% 9.5s146.4 Mb [] 1% 5.7s146.4 Mb [] 2% 3.6s146.4 Mb [] 2% 3.4s146.4 Mb [] 3% 3.1s146.4 Mb [] 4% 3.0s146.4 Mb [] 4% 4.2s146.4 Mb [] 4% 5.2s146.4 Mb [] 4% 5.5s146.4 Mb [] 4% 5.2s146.4 Mb [] 5% 4.6s146.4 Mb [] 6% 4.0s146.4 Mb [] 7% 3.8s146.4 Mb [] 8% 3.7s146.4 Mb [] 8% 3.6s146.4 Mb [] 10% 3.3s146.4 Mb [] 10% 3.4s146.4 Mb [] 11% 3.3s146.4 Mb [] 12% 3.2s146.4 Mb [] 13% 3.1s146.4 Mb [] 14% 2.9s146.4 Mb [] 16% 2.7s146.4 Mb [] 17% 2.7s146.4 Mb [] 17% 2.6s146.4 Mb [] 18% 2.6s146.4 Mb [] 19% 2.5s146.4 Mb [] 19% 2.6s146.4 Mb [] 20% 2.6s146.4 Mb [] 21% 2.5s146.4 Mb [] 22% 2.5s146.4 Mb [] 23% 2.4s146.4 Mb [] 24% 2.3s146.4 Mb [] 25% 2.3s146.4 Mb [] 26% 2.2s146.4 Mb [] 27% 2.2s146.4 Mb [] 28% 2.2s146.4 Mb [] 29% 2.1s146.4 Mb [] 31% 2.0s146.4 Mb [] 32% 1.9s146.4 Mb [] 33% 1.9s146.4 Mb [] 34% 1.9s146.4 Mb [] 35% 1.8s146.4 Mb [] 36% 1.8s146.4 Mb [] 36%

In [3]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random
import os
import numpy as np
from playwright.async_api import async_playwright

In [4]:
def match_span_class(tag):
    parent_span = tag.find_parent('span')
    if parent_span is not None:
        span_classes = parent_span.get('class', [])
        return any(cls.startswith('SummaryItemResponsive') for cls in span_classes)
    return False

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
faces = ['happiness', 'sadness', 'anger' ]

In [7]:
data_path = 'drive/MyDrive/faces/data'

In [8]:
def get_image(photo_url, path, file_name, face):
  image_response = requests.get(photo_url)
  image_content = image_response.content
  img = Image.open(BytesIO(image_content))
  if img.mode == 'RGB':
    os.makedirs(f'{path}/{face}', exist_ok=True)
    img.save(f'{path}/{face}/{file_name}', 'png')

In [9]:
def get_photos(source, path, face):
  for image in source:
    http = image.find('https://')
    if http != -1:
      file_name = image.split('/')[-1]
      get_image(image, path, file_name, face)

In [ ]:
#pages implicated by hand
images = []
for face in faces:
  modified_face = face.replace('-', '%20')
  async def run():
    async with async_playwright() as p:
      browser = await p.chromium.launch()
      page = await browser.new_page()
      await page.goto(f'https://www.istockphoto.com/pl/search/2/image?ethnicity=caucasian&numberofpeople=one&page=23&phrase={face}') #input pages from 1-100
      # Get the page content
      content = await page.content()
      await browser.close()
      # Use Beautiful Soup to parse the HTML
      soup = BeautifulSoup(content, 'html.parser')
      # Find all image tags and add their src attributes to the image_sources list
      all_images = soup.find_all('img')
      images_in_span = soup.find_all(lambda tag: tag.name == 'img' and match_span_class(tag))
      interesting_images = [img for img in all_images if img not in images_in_span]
      for img in interesting_images:
        src = img.get('src')
        images.append(src)
      get_photos(images, data_path, face)
      print(f"Downloaded {len(images)} photos.")
      print(f"{len(os.listdir(f'{data_path}/{face}'))} photos in {face} folder.")
  images = []
  await run()

# **2. Face detection**

In [11]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def face_recognition(photo, array):
  image = cv2.imread(photo)
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  face = face_classifier.detectMultiScale(gray_image, 1.3, 4)
  if len(face) > 0:
   for i, (x, y, w, h) in enumerate(face):
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
      face = image[y:y + h, x:x + w]
      array.append(face)

In [ ]:
anger = []
sadness = []
happiness = []


In [ ]:
def faces_array(array, photos, path):
  files_folder = os.listdir(f'{path}/{photos}')
  print(f'{path}/{photos}')
  for i in range(0, len(files_folder)):
    img_path = f'{path}/{photos}/{files_folder[i]}'
    face_recognition(img_path, array)

In [ ]:
faces_array(anger, "anger", data_path)
faces_array(sadness, "sadness", data_path)
faces_array(happiness, "happiness", data_path)

drive/MyDrive/faces/data/anger
drive/MyDrive/faces/data/sadness
drive/MyDrive/faces/data/happiness


In [ ]:
recognized_faces = {
    'anger': anger,
    'sadness': sadness,
    'happiness': happiness
}

# **3. Save faces**

In [ ]:
faces_path = 'drive/MyDrive/recognized_faces'

In [ ]:
from PIL import Image

for face in recognized_faces:
  os.makedirs(f'{faces_path}', exist_ok=True)
  os.makedirs(f'{faces_path}/{face}', exist_ok=True)
  directory = f'drive/MyDrive/recognized_faces/{face}'
  for i in range(0, len(recognized_faces[face])):
    img = Image.fromarray(recognized_faces[face][i])
    filename = f'{face}_{i}'
    folder = f'drive/MyDrive/recognized_faces/{face}/{filename}.jpeg'
    img.save(folder)